# Capgemini Scraper

Capgemini (IT consulting nr. 2 mener jeg) https://www.capgemini.com/

This scraper collects texts on "Artificial Intelligence" from Capgemini: https://www.capgemini.com/

## Libraries

Loading libraries...

In [2]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import re

import time

## URL List
The scraper collects urls from the returned results from searching "artificial intelligence" capgemini.com: https://www.capgemini.com/?s=artificial+intelligence. Resutls are filtered by being a "post" or an "article".

In [3]:
start_url = 'https://www.capgemini.com/?s=artificial+intelligence'

In [23]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)
driver.find_element_by_css_selector('a.section__button--cookies').click()
time.sleep(2)

pageSel = Selector(text = driver.page_source)

urldict = {}
urldict['urls'] = pageSel.css('h3.card_default__title > a::attr(href)').extract()
urldict['type'] = pageSel.css('div.card_default__content > div.card_default__type > span ::text').extract()

while True:
    try:
        driver.find_element_by_css_selector("div.pagination_show__label > a.next.page-numbers").click()
        time.sleep(3)
        
        pageSel = Selector(text = driver.page_source)
        
        types = pageSel.css('div.card_default__content > div.card_default__type > span ::text').extract()
        urls = pageSel.css('h3.card_default__title > a::attr(href)').extract()
        
        urldict['type'] = urldict['type']+ types
        urldict['urls'] = urldict['urls'] + urls
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No next button found")
        break

driver.quit()

No next button found


In [24]:
print(len(urldict['type']),
     len(urldict['urls']))

1059 1059


In [25]:
from itertools import compress

boollist = list()
for typestr in urldict['type']:
    boollist.append(typestr == 'Post' or typestr == 'Article')

urllist = list(compress(urldict['urls'], boollist))
len(urllist)

514

### Save URLs

In [26]:
#URL dump
with open('../data_raw/urls/capgemini_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

### Load URLs

In [27]:
# Load URLs
urllist = []
with open('../data_raw/urls/capgemini_urls.txt', 'r') as f:
    for line in f:
        urllist.append(line.strip())

print(len(urllist),
      '\n',
     urllist[0])

514 
 https://www.capgemini.com/2020/02/algorithmic-consent-why-informed-consent-matters-in-an-age-of-artificial-intelligence/


## Parsing single page

In [28]:
urllist[0:10]

['https://www.capgemini.com/2020/02/algorithmic-consent-why-informed-consent-matters-in-an-age-of-artificial-intelligence/',
 'https://www.capgemini.com/2020/02/us-manufacturers-slow-to-embrace-the-potential-of-artificial-intelligence/',
 'https://www.capgemini.com/2020/01/what-will-artificial-intelligence-bring-in-2020/',
 'https://www.capgemini.com/article/fundamental-impacts-in-artificial-intelligence-with-a-focus-on-ethics-in-ai/',
 'https://www.capgemini.com/2019/12/are-regional-firms-and-credit-unions-ready-to-bank-on-artificial-intelligence/',
 'https://www.capgemini.com/2019/10/artificial-intelligence-is-an-important-cybersecurity-tool-but-its-no-silver-bullet/',
 'https://www.capgemini.com/2019/08/the-ethics-of-artificial-intelligence/',
 'https://www.capgemini.com/2019/07/breaking-through-in-artificial-intelligence/',
 'https://www.capgemini.com/article/implementing-artificial-intelligence-successfully/',
 'https://www.capgemini.com/2019/02/ai-ml-breeds-new-form-capitalism/']

In [121]:
page_url = urllist[511] #3
page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)
page_url

'https://www.capgemini.com/2016/12/kickstart-my-year-7-trends-in-insights-data-for-2017/'

In [122]:
page_dict = dict()
main_dict = dict()

In [123]:
type_regex = r'(?<=\[\{\"post_type\":\")\w{2,10}(?=\",\")'
date_regex = r'(?<=\",\"date_posted\":\")\d{4}\-\d{1,2}\-\d{1,2}'

page_scripts = page_sel.css('script::text').extract()
page_scripts = ' '.join(page_scripts)

siteno = 1

page_id = 'site' + str(siteno)

page_title = page_sel.css('title::text').extract_first()
page_type = re.findall(type_regex, page_scripts)[0]
page_date = re.findall(date_regex, page_scripts)[0]
page_dldate = date.today().strftime('%Y/%m/%d')

page_text = page_sel.css('div.hero-inset__intro > p::text').extract()
page_text.append(' '.join(page_sel.css('article > div.section.container > article > div.row > div.section__block > div.article-text ::text').extract()))
page_text = ' '.join(page_text)

page_dict['url'] = page_url
page_dict['title'] = page_title
page_dict['article date'] = page_date
page_dict['download date'] = page_dldate
page_dict['text type'] = page_type
page_dict['text'] = page_text

main_dict[page_id] = page_dict

In [124]:
print(len(main_dict),
      main_dict['site1'])

1 {'url': 'https://www.capgemini.com/2016/12/kickstart-my-year-7-trends-in-insights-data-for-2017/', 'title': 'Kickstart My Year: 7 trends in Insights & Data for 2017 – Capgemini Worldwide', 'article date': '2016-12-21', 'download date': '2020/03/11', 'text type': 'post', 'text': 'Still remember our rogue kickoff for 2016 with seven key trends in the world of BI, analytics and AI? Pretty accurate they were, no? Anyway, much has changed in just a year and we have much to look forward to in 2017, with data being the undisputed driver of any serious business change effort. Just […] \n\t\t\t\t\t\n\t\t\t\t\t Still remember our  rogue kickoff for 2016  with seven key trends in the world of BI, analytics and AI? Pretty accurate they were, no? Anyway, much has changed in just a year and we have much to look forward to in 2017, with data being the undisputed driver of any serious business change effort. Just like last time, we have carried out a quick survey among our global Advisory & Architec

## Capgemini Spider

In [125]:
from scrapy.crawler import CrawlerProcess

# Create class
class capgemini_spider (scrapy.Spider):
    name = "capgemini_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
                
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        type_regex = r'(?<=\[\{\"post_type\":\")\w{2,10}(?=\",\")'
        date_regex = r'(?<=\",\"date_posted\":\")\d{4}\-\d{1,2}\-\d{1,2}'

        page_scripts = page_sel.css('script::text').extract()
        page_scripts = ' '.join(page_scripts)

        page_id = 'site' + str(siteno)

        page_title = page_sel.css('title::text').extract_first()
        page_type = re.findall(type_regex, page_scripts)[0]
        page_date = re.findall(date_regex, page_scripts)[0]
        page_dldate = date.today().strftime('%Y/%m/%d')

        page_text = page_sel.css('div.hero-inset__intro > p::text').extract()
        page_text.append(' '.join(page_sel.css('article > div.section.container > article > div.row > div.section__block > div.article-text ::text').extract()))
        page_text = ' '.join(page_text)

        page_dict['url'] = page_url
        page_dict['title'] = page_title
        page_dict['article date'] = page_date
        page_dict['download date'] = page_dldate
        page_dict['text type'] = page_type
        page_dict['text'] = page_text

        main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(capgemini_spider)
process.start()

2020-03-11 09:18:12 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2020-03-11 09:18:12 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.6.1, Platform Windows-10-10.0.18362-SP0
2020-03-11 09:18:12 [scrapy.crawler] INFO: Overridden settings: {}
2020-03-11 09:18:13 [scrapy.extensions.telnet] INFO: Telnet Password: 7fbec4948f7a5385
2020-03-11 09:18:13 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-03-11 09:18:14 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade

https://www.capgemini.com/article/fundamental-impacts-in-artificial-intelligence-with-a-focus-on-ethics-in-ai/
https://www.capgemini.com/2020/02/algorithmic-consent-why-informed-consent-matters-in-an-age-of-artificial-intelligence/


2020-03-11 09:18:14 [urllib3.connectionpool] DEBUG: https://www.capgemini.com:443 "GET /2020/02/algorithmic-consent-why-informed-consent-matters-in-an-age-of-artificial-intelligence/ HTTP/1.1" 200 24209
2020-03-11 09:18:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.capgemini.com/2019/12/are-regional-firms-and-credit-unions-ready-to-bank-on-artificial-intelligence/> (referer: None)
2020-03-11 09:18:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.capgemini.com/2019/08/the-ethics-of-artificial-intelligence/> (referer: None)
2020-03-11 09:18:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.capgemini.com/2019/07/breaking-through-in-artificial-intelligence/> (referer: None)
2020-03-11 09:18:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.capgemini.com/2020/01/what-will-artificial-intelligence-bring-in-2020/> (referer: None)
2020-03-11 09:18:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.capgemini.com/2020/02/us-manufactu

https://www.capgemini.com/2019/12/are-regional-firms-and-credit-unions-ready-to-bank-on-artificial-intelligence/
https://www.capgemini.com/2019/08/the-ethics-of-artificial-intelligence/


2020-03-11 09:18:15 [urllib3.connectionpool] DEBUG: https://www.capgemini.com:443 "GET /2019/08/the-ethics-of-artificial-intelligence/ HTTP/1.1" 200 23554
2020-03-11 09:18:15 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.capgemini.com:443


...

2020-03-11 09:21:50 [urllib3.connectionpool] DEBUG: https://www.capgemini.com:443 "GET /2017/06/three-must-have-accelerators-to-drive-your-automation-efforts/ HTTP/1.1" 200 None
2020-03-11 09:21:50 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.capgemini.com:443
2020-03-11 09:21:50 [urllib3.connectionpool] DEBUG: https://www.capgemini.com:443 "GET /2017/03/technovision-2017-preview-work-that-machine/ HTTP/1.1" 200 25779
2020-03-11 09:21:50 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.capgemini.com:443
2020-03-11 09:21:50 [urllib3.connectionpool] DEBUG: https://www.capgemini.com:443 "GET /2017/03/technovision-2017-preview-max-machina/ HTTP/1.1" 200 22663


https://www.capgemini.com/2017/03/technovision-2017-preview-work-that-machine/
https://www.capgemini.com/2017/03/technovision-2017-preview-max-machina/


2020-03-11 09:21:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.capgemini.com/2012/12/soul-of-the-machine/> (referer: None)
2020-03-11 09:21:51 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.capgemini.com:443


https://www.capgemini.com/2012/12/soul-of-the-machine/


2020-03-11 09:21:51 [urllib3.connectionpool] DEBUG: https://www.capgemini.com:443 "GET /2012/12/soul-of-the-machine/ HTTP/1.1" 200 None
2020-03-11 09:21:51 [scrapy.core.engine] INFO: Closing spider (finished)
2020-03-11 09:21:51 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 142031,
 'downloader/request_count': 514,
 'downloader/request_method_count/GET': 514,
 'downloader/response_bytes': 13234498,
 'downloader/response_count': 514,
 'downloader/response_status_count/200': 514,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 3, 11, 8, 21, 51, 716612),
 'log_count/DEBUG': 1543,
 'log_count/INFO': 11,
 'response_received_count': 514,
 'scheduler/dequeued': 514,
 'scheduler/dequeued/memory': 514,
 'scheduler/enqueued': 514,
 'scheduler/enqueued/memory': 514,
 'start_time': datetime.datetime(2020, 3, 11, 8, 18, 14, 176573)}
2020-03-11 09:21:51 [scrapy.core.engine] INFO: Spider closed (finished)


In [126]:
process.stop()

<DeferredList at 0x27150d6e860 current result: []>

In [127]:
len(main_dict)

514

In [128]:
import json

with open('..\\data_raw\\articles\\capgemini_articles.json', 'w') as file:
    json.dump(main_dict, file)